In [ ]:
import sqlite3
from sqlite3 import Error


/* QUESTIONS 
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

SELECT *
FROM Facilities
WHERE membercost > 0;

/* Q2: How many facilities do not charge a fee to members? */

SELECT COUNT(name) AS facilities_zero_charge
FROM Facilities 
WHERE membercost = 0;

/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */

SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE membercost < (monthlymaintenance * 0.2);

/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

SELECT facid, name
FROM Facilities
WHERE facid IN (1,5);

/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

SELECT 
name, 
monthlymaintenance,
	CASE 
		WHEN monthlymaintenance > 100 THEN 'expensive'
    	ELSE 'cheap'
END AS monthlymaintenancelabel
FROM Facilities;

/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

SELECT firstname, surname, joindate
FROM Members
WHERE joindate = (SELECT MAX(joindate) FROM Members);

/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

SELECT name, CONCAT(firstname, ' ', surname) AS tennis_bookings 
FROM Bookings 
INNER JOIN Facilities 
ON Bookings.facid = Facilities.facid
INNER JOIN Members 
ON Bookings.memid = Members.memid
WHERE Facilities.facid IN (0, 1) AND Members.memid IS NOT NULL 
ORDER BY surname, name;

/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

SELECT DISTINCT CONCAT( firstname, ' ', surname ) AS customer_name, name,
	CASE b.memid
		WHEN 0 THEN guestcost * slots
		ELSE membercost * slots
	END AS m_cost
FROM Bookings AS b
INNER JOIN Members AS m ON b.memid = m.memid
INNER JOIN Facilities AS f ON b.facid = f.facid
WHERE starttime LIKE '2012-09-14%'
AND guestcost * slots >30
OR membercost * slots >30
ORDER BY m_cost DESC

/* Q9: This time, produce the same result as in Q8, but using a subquery. */

SELECT CONCAT(customer_name) AS customer_name,
    name,
    m_cost
FROM (
    SELECT 
        CONCAT(m.firstname, ' ', m.surname) AS customer_name,
        f.name,
        CASE b.memid
            WHEN 0 THEN f.guestcost * b.slots
            ELSE f.membercost * b.slots
        END AS m_cost
FROM Bookings AS b
INNER JOIN Members AS m ON b.memid = m.memid
INNER JOIN Facilities AS f ON b.facid = f.facid
WHERE starttime LIKE '2012-09-14%'
AND (f.guestcost * b.slots > 30 OR f.membercost * b.slots > 30)
) AS subquery
ORDER BY 
    m_cost DESC;

In [ ]:
QUESTIONS:
/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

SELECT b.facid, f.name, SUM(
    CASE
        WHEN b.memid = 0 THEN f.guestcost * b.slots
        ELSE f.membercost * b.slots
    END
) AS total_revenue
FROM Bookings AS b
INNER JOIN Members AS m ON b.memid = m.memid
INNER JOIN Facilities AS f ON b.facid = f.facid
WHERE total_revenue < 1000
GROUP BY f.name;

/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

SELECT
    m.memid AS memberid,
    m.firstname || ' ' || m.surname AS member_name,
    sub.firstname || ' ' || sub.surname AS recommender_name
FROM Members AS m
JOIN Members AS sub ON m.recommendedby = sub.memid
WHERE m.recommendedby IS NOT NULL
ORDER BY recommender_name;

/* Q12: Find the facilities with their usage by member, but not guests */

SELECT DISTINCT firstname, surname, b.memid, b.facid, name 
    FROM Bookings AS b INNER JOIN Facilities AS f ON b.facid = f.facid 
    INNER JOIN Members AS m ON b.memid = m.memid 
    WHERE b.memid !=0 GROUP BY b.facid"

/* Q13: Find the facilities usage by month, but not guests */

SELECT EXTRACT(
MONTH FROM starttime ) AS
MONTH , facid, SUM( slots ) AS usage_total
FROM Bookings
GROUP BY MONTH , facid